# Arizona FARMS Agrivoltaic Test Site Irradiance Modeling

Author: Kai Lepley
Origin: November 17, 2023
Updated: November 17, 2023

Here, I will be modeling the University of Arizona FARMS field site, managed by AES near Casa Grande, Arizona. The site is a 121 MW PV site composed of single-axis tracking panels with an 18-foot row spacing.

Specs:<br>
Row-spacing: 20' = 6096mm<br>
Panel: First Solar Series 6 435W (thin-film monofacial)<br>
Panel length: 2009mm<br>
Panel width: 1232mm<br>
Panel orientation: 1P<br>
Tracking: single-axis east-west<br>
Grade to top of pile: 4' 11" = 1498.6mm<br>
Top of pile to torque tube: 6" = 152.4mm<br>
Module to grade: 30" = 762mm<br>
Max tracker tiler: 52°<br>
GCR: panel length / row-spacing = 2009mm / 6096mm = 0.33<br>

Assuming 1-meter panel height. This should be updated with more accuract specifications later.

Kai's NSRDB API Key: BY01WmafAijSce07CkJFJpsyBSzuEEJHmHSrlNkU

In [1]:
NREL_API_KEY = 'BY01WmafAijSce07CkJFJpsyBSzuEEJHmHSrlNkU'  # <-- please set your NREL API key here
# note you must use "quotes" around your key as it is a string.

if NREL_API_KEY is None:
       NREL_API_KEY = 'DEMO_KEY'  # OK for this demo, but better to get your own key

In [2]:
import pvlib

metdata, metadata = pvlib.iotools.get_psm3(
    latitude=32.852182, longitude=-111.554986,
    api_key=NREL_API_KEY,
    email='kai.lepley@nrel.gov',
    names='tmy', map_variables=True)
metadata

{'Source': 'NSRDB',
 'Location ID': '73012',
 'City': '-',
 'State': '-',
 'Country': '-',
 'Time Zone': -7,
 'Local Time Zone': -7,
 'Dew Point Units': 'c',
 'DHI Units': 'w/m2',
 'DNI Units': 'w/m2',
 'GHI Units': 'w/m2',
 'Temperature Units': 'c',
 'Pressure Units': 'mbar',
 'Wind Direction Units': 'Degrees',
 'Wind Speed Units': 'm/s',
 'Surface Albedo Units': 'N/A',
 'Version': '3.2.0',
 'latitude': 32.85,
 'longitude': -111.54,
 'altitude': 442}

In [3]:
from bifacial_radiance import *
import numpy as np

#bifacial_radiance.__version__

In [4]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'TEMP' /  'FARMS_Arizona')

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in /Users/klepley/Documents/GitHub/InSPIRE/TEMP/FARMS_Arizona


In [5]:
# Create a RadianceObj 'object' named bifacial_example. no whitespace allowed
radObj = RadianceObj('FARMS_Arizona', path = str(testfolder))

albedo = 0.25
radObj.setGround(albedo)

path = /Users/klepley/Documents/GitHub/InSPIRE/TEMP/FARMS_Arizona
Loading albedo, 1 value(s), 0.250 avg
1 nonzero albedo values.


In [6]:
# Some of the names changed internally. While bifacial_radiance updates their expected names, we are renaming the values here
metadata['timezone'] = metadata['Time Zone']
metadata['county'] = 'Pinal'
metadata['elevation'] = metadata['altitude']
metadata['state'] = metadata['State']
metadata['country'] = metadata['Country']
metdata['Albedo'] = metdata['albedo']

In [7]:
#starttime can be 'MM_DD', or 'MM_DD_HH'
#metData = radObj.NSRDBWeatherData(metadata, metdata, starttime='03_10_21', endtime='11_21_21',coerce_year=2021)

# Above was working, but suddenly began giving "no attribute NSRDBWeatherData", so trying EPW
# Pull in meteorological data using pyEPW for any global lat/lon
epwfile = radObj.getEPW(lat = 32.852182, lon = -111.554986)  # Coordinates of array
# Read in the weather data pulled in above.
metdata = radObj.readWeatherFile(weatherFile = epwfile)

Getting weather file: USA_AZ_Casa.Grande.AWOS.722748_TMY3.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


In [8]:
metdata.datetime  # printing the contents of metData to see how many times got loaded.

[Timestamp('2021-01-01 09:00:00-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2021-01-01 10:00:00-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2021-01-01 11:00:00-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2021-01-01 12:00:00-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2021-01-01 13:00:00-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2021-01-01 14:00:00-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2021-01-01 15:00:00-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2021-01-01 16:00:00-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2021-01-01 17:00:00-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2021-01-02 09:00:00-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2021-01-02 10:00:00-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2021-01-02 11:00:00-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2021-01-02 12:00:00-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2021-01-02 13:00:00-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2021-01-02 14:00:00-07

In [9]:
limit_angle = 52 # Max tracker angle is 52 degrees
angledelta = 2 # Sampling every 2 degrees?
backtrack = True
gcr = 0.33
cumulativesky = True # This is important for this example!
trackerdict = radObj.set1axis(metdata = metdata, limit_angle = limit_angle, backtrack = backtrack,
                            gcr = gcr, cumulativesky = cumulativesky)

Saving file EPWs/1axis_-50.0.csv, # points: 617
Saving file EPWs/1axis_-45.0.csv, # points: 177
Saving file EPWs/1axis_-40.0.csv, # points: 156
Saving file EPWs/1axis_-35.0.csv, # points: 96
Saving file EPWs/1axis_-30.0.csv, # points: 243
Saving file EPWs/1axis_-25.0.csv, # points: 176
Saving file EPWs/1axis_-20.0.csv, # points: 99
Saving file EPWs/1axis_-15.0.csv, # points: 289
Saving file EPWs/1axis_-10.0.csv, # points: 89
Saving file EPWs/1axis_-5.0.csv, # points: 172
Saving file EPWs/1axis_-0.0.csv, # points: 224
Saving file EPWs/1axis_5.0.csv, # points: 93
Saving file EPWs/1axis_10.0.csv, # points: 4
Saving file EPWs/1axis_15.0.csv, # points: 210
Saving file EPWs/1axis_20.0.csv, # points: 105
Saving file EPWs/1axis_25.0.csv, # points: 157
Saving file EPWs/1axis_30.0.csv, # points: 243
Saving file EPWs/1axis_35.0.csv, # points: 111
Saving file EPWs/1axis_40.0.csv, # points: 139
Saving file EPWs/1axis_45.0.csv, # points: 151
Saving file EPWs/1axis_50.0.csv, # points: 650


In [10]:
trackerdict = radObj.genCumSky1axis()

message: There were 617 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies/1axis_-50.0.rad
message: There were 177 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies/1axis_-45.0.rad
message: There were 156 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies/1axis_-40.0.rad
message: There were 96 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies/1axis_-35.0.rad
message: There were 243 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies/1axis_-30.0.rad
message: There were 175 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies/1axis_-25.0.rad
message: There were 99 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies/1axis_-20.0.rad
message: There were 289 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies/1axis_-15.0.rad
message: There were 89 sun up hours in thi

In [11]:
# Create module
x = 1.232  # meters
y = 2.009  # meters
moduletype = 'test-module'
numpanels = 1
zgap = 0.05
ygap = 0.10
xgap = 0.02

module = radObj.makeModule(name=moduletype, x=x, y=y,xgap=xgap, ygap=ygap, zgap=zgap,
                numpanels=numpanels)

module.addTorquetube(diameter=0.1, material='Metal_Grey', tubetype='round') # New torquetube generation function
print()
print(module)
print()
print(module.torquetube)


Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects/test-module.rad will be overwritten

Module test-module updated in module.json
Pre-existing .rad file objects/test-module.rad will be overwritten


{'x': 1.232, 'y': 2.009, 'z': 0.02, 'modulematerial': 'black', 'scenex': 1.252, 'sceney': 2.009, 'scenez': 0.1, 'numpanels': 1, 'bifi': 1, 'text': '! genbox black test-module 1.232 2.009 0.02 | xform -t -0.616 -1.0045 0.1 -a 1 -t 0 2.109 0\r\n! genrev Metal_Grey tube1 t*1.252 0.05 32 | xform -ry 90 -t -0.626 0 0', 'modulefile': 'objects/test-module.rad', 'glass': False, 'offsetfromaxis': 0.1, 'xgap': 0.02, 'ygap': 0.1, 'zgap': 0.05}

{'diameter': 0.1, 'tubetype': 'round', 'material': 'Metal_Grey', 'visible': True}


In [12]:
# Create the scene
hub_height = 1.651 # height of panel tracker hub
pitch = 6.096 # inter-row spacing
sceneDict = {'gcr': gcr,'hub_height':hub_height, 'nMods': 20, 'nRows': 7}

In [13]:
# Create the scene
trackerdict = radObj.makeScene1axis(trackerdict = trackerdict, module = module, sceneDict = sceneDict)


Making .rad files for cumulativesky 1-axis workflow
21 Radfiles created in /objects/


In [14]:
# Combine ground, sky, and scene objects
trackerdict = radObj.makeOct1axis(trackerdict = trackerdict)


Making 21 octfiles in root directory.
Created 1axis_-50.0.oct
Created 1axis_-45.0.oct
Created 1axis_-40.0.oct
Created 1axis_-35.0.oct
Created 1axis_-30.0.oct
Created 1axis_-25.0.oct
Created 1axis_-20.0.oct
Created 1axis_-15.0.oct
Created 1axis_-10.0.oct
Created 1axis_-5.0.oct
Created 1axis_-0.0.oct
Created 1axis_5.0.oct
Created 1axis_10.0.oct
Created 1axis_15.0.oct
Created 1axis_20.0.oct
Created 1axis_25.0.oct
Created 1axis_30.0.oct
Created 1axis_35.0.oct
Created 1axis_40.0.oct
Created 1axis_45.0.oct
Created 1axis_50.0.oct


In [21]:
# Visualize the system
#get_ipython().system('rpict -vf views/front.vp -e .01 1axis_20.0.oct > scene.hdr')
#rpict -vp 10 5 3 -vd 1 -.5 0 scene.oct > scene.jpg

#get_ipython().system('rvu -vf /Users/klepley/Documents/GitHub/InSPIRE/TEMP/FARMS_Arizona/views/front.vp -e 1axis_20.0.oct')

In [15]:
# Analysis
modWanted = 9
rowWanted = 2
customname = '_Row_2_Module_09' # This is useful if we want to do various analysis.
trackerdict = radObj.analysis1axis(trackerdict, modWanted=9, rowWanted = 2, customname=customname)

Linescan in process: 1axis_-50.0_Row_2_Module_09_Front
Linescan in process: 1axis_-50.0_Row_2_Module_09_Back
Saved: results/irr_1axis_-50.0_Row_2_Module_09.csv
Index: -50.0. Wm2Front: 317817.8851851852. Wm2Back: 26996.4762962963
Linescan in process: 1axis_-45.0_Row_2_Module_09_Front
Linescan in process: 1axis_-45.0_Row_2_Module_09_Back
Saved: results/irr_1axis_-45.0_Row_2_Module_09.csv
Index: -45.0. Wm2Front: 98488.62037037036. Wm2Back: 10028.43988888889
Linescan in process: 1axis_-40.0_Row_2_Module_09_Front
Linescan in process: 1axis_-40.0_Row_2_Module_09_Back
Saved: results/irr_1axis_-40.0_Row_2_Module_09.csv
Index: -40.0. Wm2Front: 95365.45888888888. Wm2Back: 9685.130222222222
Linescan in process: 1axis_-35.0_Row_2_Module_09_Front
Linescan in process: 1axis_-35.0_Row_2_Module_09_Back
Saved: results/irr_1axis_-35.0_Row_2_Module_09.csv
Index: -35.0. Wm2Front: 46570.26. Wm2Back: 5119.0794074074065
Linescan in process: 1axis_-30.0_Row_2_Module_09_Front
Linescan in process: 1axis_-30.0_R